# Examples using the Spectral mixture kernel for single channel

* Mauna Loa
* Airline passengers

In [ ]:
# import library if it is not installed
import sys
sys.path.insert(0, '../')

import mogptk
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%reload_ext autoreload
%autoreload 2

In [ ]:
sns.set_context('notebook', font_scale=1.4)
sns.set_style('ticks')
plt.rcParams['figure.figsize'] = (10, 4)

# Mauna Loa

In [ ]:
# dataset mauna loa
from sklearn.datasets import fetch_openml

def load_mauna_loa_atmospheric_co2():
    ml_data = fetch_openml(data_id=41187)
    months = []
    ppmv_sums = []
    counts = []

    y = ml_data.data[:, 0]
    m = ml_data.data[:, 1]
    month_float = y + (m - 1) / 12
    ppmvs = ml_data.target

    for month, ppmv in zip(month_float, ppmvs):
        if not months or month != months[-1]:
            months.append(month)
            ppmv_sums.append(ppmv)
            counts.append(1)
        else:
            # aggregate monthly sum to produce average
            ppmv_sums[-1] += ppmv
            counts[-1] += 1

    months = np.asarray(months).reshape(-1)
    avg_ppmvs = np.asarray(ppmv_sums) / counts
    return months, avg_ppmvs

In [ ]:
# load dataset
x, y = load_mauna_loa_atmospheric_co2()

stop = 200

# plot
plt.plot(x[:stop], y[:stop], label='Train')
plt.plot(x[stop:], y[stop:], 'r', label='Test')
plt.legend()
plt.xlabel('Year')
plt.ylabel('CO2');
plt.title('Mauna Loa dataset');

In [ ]:
# add data
data = mogptk.Data(x[:stop, np.newaxis], y[:stop])
data.transform(mogptk.TransformDetrend(2))

# create model
model = mogptk.SM([data], Q=10)

model.plot_psd(title='PSD with random parameters')

In [ ]:
# plot detrended data
data.plot(title='Detrended Data')

In [ ]:
method = 'BNSE'
model.init_params(method)
model.plot_psd(title='PSD with {} initialization'.format(method))

In [ ]:
model.train(method='L-BFGS-B', maxiter=2000, tol=1e-20)
model.plot_psd(title='PSD with model trained')

In [ ]:
y_pred, lower_ci, upper_ci = model.predict(x)
fig, ax = model.plot_prediction(title='Mauna Loa')
ax.set_xlabel('Year')
ax.set_ylabel('CO2');

# Airplane passangers

In [ ]:
air = np.loadtxt('data/Airline_passenger.csv')

stop = 96

x = air[: ,0]
y = air[:, 1]

# add data
data = mogptk.Data(x[:stop, np.newaxis], y[:stop])

data.transform(mogptk.TransformLog)
data.transform(mogptk.TransformDetrend(2))

# create model
model = mogptk.SM([data], Q=10)

data.plot()

In [ ]:
method = 'BNSE'
model.init_params(method)

model.plot_psd(title='PSD with {} initialization'.format(method))

In [ ]:
model.train('L-BFGS-B', maxiter=2000, tol=1e-80)

model.plot_psd(title='PSD with model trained')

In [ ]:
model.predict(x)
fig, ax = model.plot_prediction(title='Mauna Loa')
ax.set_xlabel('Month')
ax.set_ylabel('Passengers');